This file aims to construct a pairwise paired-t and welch table for comparison of systems of a simulation problem





In [1]:

from scipy import stats
import pandas as pd
import numpy as np
import math

In [6]:
def confidence_intervals(x,y,name, alpha = 0.0076923):
    

    #WELCH
    ## Welch-Satterthwaite Degrees of Freedom ##
    dof = (x.var()/x.size + y.var()/y.size)**2 / ((x.var()/x.size)**2 / (x.size-1) + (y.var()/y.size)**2 / (y.size-1))
    lower_welch =x.mean()-y.mean()-stats.t.ppf(1-alpha/2, dof)*math.sqrt((x.var()/x.size) +(y.var()/y.size))
    upper_welch =x.mean()-y.mean()+stats.t.ppf(1-alpha/2, dof)*math.sqrt((x.var()/x.size) +(y.var()/y.size))

    results_welch = np.array(lower_welch, upper_welch)

    #PAIRED - T
    paired_t_data = x - y #simulation - baseline
    half_width = (stats.t.ppf(1-alpha/2, paired_t_data.size)*paired_t_data.var())/np.sqrt(paired_t_data.size)
    barbar = x.mean() - y.mean()
    lower_paired_t = barbar - half_width
    upper_paired_t = barbar + half_width

    results_paired_t = np.array(lower_paired_t, upper_paired_t)

    #return the two results arrays
    return [results_welch, results_paired_t]




In [3]:
systems = ["(3,2)Comparative new", "(3,2)Comparative upgrade", "(3,3)Comparative new", "(3,3)Comparative upgrade",
               "(4,2)Comparative new","(4,2)Comparative upgrade","(4,3)Comparative new","(4,3)Comparative upgrade",
               "(5,2)Comparative new","(5,2)Comparative upgrade","(5,3)Comparative new","(5,3)Comparative upgrade"]


In [9]:
pairwise_welch = pd.DataFrame(columns = systems, index= systems)
pairwise_welch = pairwise_welch.fillna(0)
pairwise_paired_t = pd.DataFrame(columns = systems)
for x_name in systems:
    comp_systems = systems.copy()
    comp_systems.remove(x_name)
    x = pd.read_csv("output/"+x_name+"/performance.csv")
    x = x['objective function']
    for y_name in comp_systems:
        y = pd.read_csv("output/"+y_name+"/performance.csv")
        y = y["objective function"]
        name = x_name+" vs "+y_name
        results = confidence_intervals(x,y, name)

        results_welch = str(results[0])
        pairwise_welch.at[x_name, y_name] = results_welch
        print(pairwise_welch)

KeyboardInterrupt: 